In [1]:
import pandas as pd
import os
import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(mpl.__version__)
# seaborn 스타일 사용
plt.style.use('seaborn-notebook')

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


3.4.3


In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [19]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

# 파이캐럿 (전처리 X)

In [38]:
pycaret_data = pd.get_dummies(data = train, columns = ['Gender'], prefix = 'Gender')

In [44]:
from pycaret.classification import *
from pycaret.regression import *

In [45]:
setup_clf = setup(data = pycaret_data, target = 'Target', fold_shuffle=True, use_gpu=True, silent=True)

,Description,Value
0,session_id,4968
1,Target,Target
2,Original Data,"(1253, 11)"
3,Missing Values,False
4,Numeric Features,10
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(877, 10)"


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

In [46]:
top10 = compare_models(sort='MAE', n_select=10, fold = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,1.5939,5.3799,2.3059,0.5006,0.1897,0.1530,0.0220
lr,Linear Regression,1.6293,5.2335,2.2775,0.5120,0.1863,0.1640,0.0100
br,Bayesian Ridge,1.6303,5.2233,2.2753,0.5131,0.1863,0.1642,0.0060
lar,Least Angle Regression,1.6304,5.2341,2.2776,0.5120,0.1864,0.1642,0.0080
ridge,Ridge Regression,1.6546,5.4147,2.3168,0.4958,0.1908,0.1673,0.0040
rf,Random Forest Regressor,1.6549,5.5670,2.3503,0.4788,0.1879,0.1627,0.4560
catboost,CatBoost Regressor,1.6593,5.6168,2.3624,0.4724,0.1868,0.1618,0.9820
knn,K Neighbors Regressor,1.6632,5.5531,2.3485,0.4807,0.1892,0.1632,0.2320
gbr,Gradient Boosting Regressor,1.6967,5.7511,2.3930,0.4592,0.1900,0.1664,0.0780
et,Extra Trees Regressor,1.7017,5.6755,2.3722,0.4676,0.1918,0.1691,0.4540


In [47]:
tuned_top10 = [tune_model(i, fold = 5) for i in top10]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.5748,4.2657,2.0654,0.5179,0.1719,0.1598
1,1.6230,4.6553,2.1576,0.5281,0.1795,0.1630
2,1.5001,5.2743,2.2966,0.5206,0.1774,0.1428
3,1.7212,6.2700,2.5040,0.4043,0.1910,0.1654
4,1.7933,6.9710,2.6403,0.4691,0.2104,0.1774
Mean,1.6425,5.4873,2.3328,0.4880,0.1860,0.1617
SD,0.1041,1.0044,0.2132,0.0467,0.0137,0.0112


In [48]:
blender_top10 = blend_models(estimator_list=tuned_top10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.2993,3.1113,1.7639,0.5737,0.1510,0.1358
1,1.5768,4.1443,2.0358,0.5992,0.1683,0.1600
2,1.6797,4.5528,2.1337,0.5728,0.1764,0.1672
3,1.5538,4.0078,2.0020,0.5561,0.1730,0.1609
4,1.5409,5.6484,2.3766,0.5202,0.1789,0.1357
5,1.3615,4.2971,2.0729,0.5589,0.1669,0.1429
6,1.5907,6.7134,2.5910,0.3519,0.1860,0.1468
7,1.5701,4.3976,2.0971,0.5873,0.1716,0.1542
8,1.7428,7.4002,2.7203,0.4430,0.2167,0.1665
9,1.6638,5.5984,2.3661,0.5731,0.2011,0.1762


In [49]:
final_model = finalize_model(blender_top10)
prediction = predict_model(final_model, data= test)

In [50]:
prediction

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M,Label
0,0.595,0.470,0.155,1.1210,0.4515,0.1780,0.1550,1,0,0,10.036983
1,0.580,0.450,0.150,0.9270,0.2760,0.1815,0.3600,0,0,1,12.993869
2,0.260,0.205,0.070,0.0970,0.0415,0.0190,0.0305,0,1,0,5.894238
3,0.590,0.460,0.130,1.1020,0.4550,0.2055,0.3300,0,0,1,11.238266
4,0.595,0.465,0.140,1.1130,0.5175,0.2440,0.3050,1,0,0,9.974034
...,...,...,...,...,...,...,...,...,...,...,...
2919,0.170,0.105,0.035,0.0340,0.0120,0.0085,0.0050,0,1,0,4.322134
2920,0.435,0.345,0.115,0.4180,0.2220,0.0735,0.1060,0,1,0,7.562817
2921,0.570,0.450,0.135,0.7940,0.3815,0.1415,0.2450,0,1,0,9.405466
2922,0.460,0.350,0.120,0.4885,0.1930,0.1050,0.1550,0,1,0,8.978355


In [51]:
prediction['Label']

0       10.036983
1       12.993869
2        5.894238
3       11.238266
4        9.974034
          ...    
2919     4.322134
2920     7.562817
2921     9.405466
2922     8.978355
2923    11.660644
Name: Label, Length: 2924, dtype: float64

In [52]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = prediction['Label']

submission

submission.to_csv('submission.csv',index=False)

# 실험실